In [58]:
#Le spectral clustering de Nvidia est un spectral clustering en utilisant la matrice laplacienne Lrw

In [59]:
using Random
using KrylovKit

In [60]:
n=25
X = Random.rand(1:25,n,n)
#X = Xb'.*Xb

25×25 Matrix{Int64}:
  6  15  24  24   3  18  19   2  14  …   8  14   2   5   1   2   2   5   8
  5   4  16  11   4   9  15   8   2     21  24   3   4  13   5  18   6  12
  4   5   4  12  15   2  21   7  20     16  21   6  24  11  24  22   9   6
 12   3  16  11   8  11  11   5  11     16  24   3  19   1  10  20  22  10
 11   6  17   5   5  11   5   2  15     18  19   1  22  22   3  14  25  24
  3   2  25  10  25  15  16  22  12  …   8  25  24  14  21  14   1  13  22
  4  18   9  16  23  15   2   2  16     19   6  15   6   6  21  16   6  21
  5  19   2   9  22   3  13   4  20     24  19  19  20   4  24  14   8   2
 11  24  18   4   4  11  13  24  13     11   3  12  24  10   2  17  21   7
 17  20   9  11   8   4  25  11  15     10   5  21  22  17   9  14  15  17
  7  11  16   3  13  15   3  21  17  …   1   2   3  12  13   4  25   8   1
  1   9   3  11  22  16   4  25  14      2  20  10   5   2  10   6   4  20
  5  19   3   1  12  12   5   2  13     15   7  23  13  14   6   9  14  11
  6 

In [61]:
function createD(W)
    lines,column = size(W)
    output = zeros(Float64,size(W))
    for i = 1:lines
        sum = 0
        for j = 1:column
            sum += W[i,j]
        end
        output[i,i] = sum
    end
    return output
end

createD (generic function with 1 method)

In [62]:
function euclideanDistance(p,c)
    sum = 0
    for (index,coord) = enumerate(p) 
        sum += (coord-c[index])^2
    end
    return(sqrt(sum))
end

euclideanDistance (generic function with 1 method)

In [63]:
#Cette fonction construit la matrice de similarité entièrement connnectée. Sigma est la taille du voisinage autorisé
#On utilisera la fonction de similarité gaussienne
function constructFully(X,sigma)
    lines,column = size(X)
    S = zeros(Float64,lines,column)
    for i = 1 : lines
        for j = 1 : column
            S[i,j] = exp(-euclideanDistance(X[i],X[j])^2/(2*sigma^2))
        end
    end
    return S
end
                
            

constructFully (generic function with 1 method)

In [66]:
function partioning(X,k)
    S = constructFully(X,3)
    #On prendra B = D
    B = createD(X)
    L = B-X
    (eigval,eigvecs,info) = eigsolve(L,k,:SR)
    eigval = eigval[1:k]
    for i =1:length(eigval)
        eigval[i] = eigval[i]/B[i,i]
    end
    return eigvecs[1:k]
end
partioning(X,3)

3-element Vector{Vector{ComplexF64}}:
 [-0.19999999999999998 + 0.0im, -0.19999999999999996 + 0.0im, -0.19999999999999993 + 0.0im, -0.19999999999999993 + 0.0im, -0.1999999999999999 + 0.0im, -0.1999999999999999 + 0.0im, -0.2000000000000001 + 0.0im, -0.20000000000000007 + 0.0im, -0.19999999999999998 + 0.0im, -0.20000000000000012 + 0.0im  …  -0.19999999999999987 + 0.0im, -0.2 + 0.0im, -0.19999999999999987 + 0.0im, -0.19999999999999996 + 0.0im, -0.19999999999999996 + 0.0im, -0.19999999999999998 + 0.0im, -0.2 + 0.0im, -0.19999999999999993 + 0.0im, -0.19999999999999996 + 0.0im, -0.19999999999999987 + 0.0im]
 [0.39538170111352544 + 0.0im, -0.12868507773385066 + 0.0im, 0.1437883183396267 + 0.0im, 0.3124711746878325 + 0.0im, -0.07034634346894819 + 0.0im, -0.004544447062046837 + 0.0im, 0.08892013625310642 + 0.0im, -0.058420273291297856 + 0.0im, -0.10281838959996417 + 0.0im, -0.00212232129155435 + 0.0im  …  0.09589066950337184 + 0.0im, 0.09405263187244318 + 0.0im, 0.11627710821687201 + 0.0im, -0.0